In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings

In [5]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

SEED = 13
np.random.seed(SEED)

In [ ]:
X_train,X_test, y_train,y_test = train_test_split(X,y,stratify=y,random_state=SEED,test_size=0.25)

In [3]:
def run_models():
    import time 
    
    classifiers = [SVC(random_state=SEED),
                  GaussianNB(), BernoulliNB(),
                  KNeighborsClassifier()]

    fitted_clf_lis = []
    for clf in classifiers:
        start = time.time()
        
        fitted_clf = clf.fit(X_train,y_train)
        
        y_pred = fitted_clf.predict(X_test)
        
        fitted_clf_lis.append(fitted_clf)
        
        time_taken = time.time()-start

        acc = accuracy_score(y_test,y_pred)
        prec = precision_score(y_test,y_pred,average='macro')
        rec = recall_score(y_test,y_pred,average='macro')

        print(f'{type(clf).__name__} has taken {time_taken} seconds and test scores are as follows:\n')
        print(f'Accuracy: {acc}\nPrecision: {prec}\nRecall:{rec}\n','*'*20)
        
    return(fitted_clf_lis)